# Create [COMARCS](http://stev.oapd.inaf.it/atm/lrspe.html) sed grids for speedysedfit by using the R=200 spectra 

- Download the R=200 spectra from [COMARCS web](http://stev.oapd.inaf.it/atm/lrspe.html)

In [3]:
from astropy.io import fits
import matplotlib.pyplot as plt
import joblib
import numpy as np
from glob import glob
from speedysedfit.comarcsgrid import read_comarcs_spec
from speedysedfit.broadsed import vgconv
from tqdm import tqdm
import os

In [4]:
def get_primaryhdu(REF='COMARCS'):
    n = np.zeros((1, 2), dtype=np.int32)
    primary_hdu = fits.PrimaryHDU(n)
    header = primary_hdu.header
    header['REF'] =(REF, 'Table reference')
    header['WAVUNIT'] = ('angstrom', 'wavelength units')
    header['FLXUNIT'] = ('erg/s/cm2/A', 'flux units')
    return primary_hdu

def get_tablehdu(fname, R =200):
    '''
    R: [float] is resolution
    '''
    wave, flux_norm, nuLnu, fmean, flam = read_comarcs_spec(fname)
    wave = np.array(wave, dtype=np.float32)
    flam = np.array(flam, dtype=np.float32)
    extname = os.path.basename(fname)
    paras = extname.split('_')
    TEFF = np.float(paras[1][1:])
    LOGG = np.float(paras[2][1:])/100
    M = np.float(paras[3][1:])/100
    OH = np.float(paras[4][1:])/np.float(f'1.e{len(paras[4][2:])}')
    CH = np.float(paras[5][1:])/np.float(f'1.e{len(paras[5][2:])}')
    FeH = np.float(paras[6][2:])/np.float(f'1.e{len(paras[6][3:])}')
    for para in paras[7:]:
        if 'xi' in para:
            Xi = np.float(para[2:])/np.float(f'1.e{len(para[3:])}')
    cw = fits.Column(name='wavelength', array=wave, format='E')
    cf = fits.Column(name='flux', array=flam, format='E')
    table_hdu = fits.BinTableHDU.from_columns([cw,cf])
    header = table_hdu.header
    header['EXTNAME1'] = (extname[:29], 'name of the extension') 
    header['EXTNAME2'] = (extname[29:-4], 'name of the extension')
    header['TEFF']    = (TEFF, 'Effective temperature (K)')                
    header['LOGG']    = (LOGG, 'Log(g)')
    header['OH']      = (OH, 'log(eps(O)/eps(H)) + 12')
    header['CH']      = (CH, 'log(eps(C)/eps(H)) + 12')
    header['FeH']      = (FeH, 'log(eps(Fe)/eps(H)) + 12')
    header['Vt']    = (Xi, 'microturbulence velocity (km/s)')
    table_hdu.header = header
    return table_hdu

In [6]:
dire_comarcs = '/share/lijiao/COMARCS/lrspec/isp'

filelist = glob(os.path.join(dire_comarcs, 'mxcom11v06_t3744_g+003_m0079_o78029_c75429_fe65630_n+100_xi25_c10b.isp'))
fname = filelist[0]

In [7]:
filelist = glob(os.path.join(dire_comarcs, '*.isp'))
dirout = '/share/lijiao/speedyfit/modelgrids/'
primary_hdu = get_primaryhdu(REF='CM')
table_hdus = []
#for fname in filelist:
#    print(fname)
#    table_hdu = get_tablehdu(fname)
#    table_hdus.append(table_hdu)
table_hdus = joblib.Parallel(n_jobs=10, )(joblib.delayed(get_tablehdu)(fname, R =200) for fname in tqdm(filelist[:30]))

    
#hdul = fits.HDUList([primary_hdu] + table_hdus)
#hdul.writeto(dirout+'COMARCS.fits', overwrite=True)

100%|██████████| 30/30 [00:00<00:00, 37.63it/s]


In [6]:
    
#hdul = fits.HDUList([primary_hdu] + table_hdus)
#hdul.writeto(dirout+'COMARCS.fits', overwrite=True)

In [12]:
hdul = fits.HDUList([primary_hdu] + table_hdus[:20])
hdul.writeto(dirout+'COMARCS_subset.fits', overwrite=True)

In [16]:
hdul = fits.open(dirout+'COMARCS.fits')
xis = []
for _hdul in hdul[1:]:
    xis.append(_hdul.header['VT'])

In [13]:
hdulsss = fits.open(dirout+'COMARCS_subset.fits')

In [15]:
hdulsss.close()

# check re.compiler

In [18]:
fname = '/share/lijiao/TLUSTY_grid/wangluqian/TlustyB/BG15000g475v2.flux'

line_ttt = '5.50794208D+16 2.5798-101 2.5798+101\n'
line_ttt = '5.50794208D+16 2.5798+101\n'
l0= re_dbl_fort.sub(r'\1e\2', line_ttt)
l = re_dbl_fort1.sub(r'\1e-\2', l0)
ll = re_dbl_fort2.sub(r'\1e+\2', l)
print(line_ttt)
ll

5.50794208D+16 2.5798+101



'5.50794208e+16 2.5798e+101\n'